In [28]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [29]:
## Load the Trained models

model=load_model('model.h5')

## Load encoder and scaler

with open('gender_encoder.pkl', 'rb') as file:
    gender_encoder=pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    geo_encoder=pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)



In [30]:
input_data = {
    'CreditScore': [600],
    'Geography': ['France'],
    'Gender': ['Male'],
    'Age': [40],
    'Tenure': [30],
    'Balance': [60000],
    'NumOfProducts': [2],
    'HasCrCard': [1],
    'IsActiveMember': [1],
    'EstimatedSalary': [50000]
}

input_df = pd.DataFrame(input_data)

input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,30,60000,2,1,1,50000


In [31]:
## apply gender encoder to the input data

input_df['Gender']=gender_encoder.transform(input_df['Gender'])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,30,60000,2,1,1,50000


In [32]:
## apply geo_encoder to the input data
# geo_encoder.get_feature_names_out([input_df['Geography']])

In [33]:
encoded_geography = geo_encoder.transform([input_df['Geography']])
encoded_geography.toarray()

/Users/e151858/Desktop/Projects/Personal/ann_classification/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [34]:
encoded_geography_df = pd.DataFrame(encoded_geography.toarray(), columns=geo_encoder.get_feature_names_out(['Geography']))
encoded_geography_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [35]:
## concatenate the encoded geo df and drop the original

encoded_input_df = pd.concat((input_df.drop('Geography', axis=1), encoded_geography_df), axis=1)
encoded_input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,30,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
## apply scaler to transform/normalize the inputs
scaled_input_df = scaler.transform(encoded_input_df)
scaled_input_df

array([[-0.53598516,  0.91324755,  0.10479359,  8.67426486, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [39]:
## Make prediction using the trained model

prediction = model.predict(scaled_input_df)
prediction_prob = prediction[0][0]
prediction_prob

1/1 [==============================] - 0s 11ms/step


0.00497744

In [38]:
if prediction[0][0]>=0.5:
    print(1)
elif  prediction[0][0]<0.5:
    print(0)

0
